In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# Core
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.4)
from imblearn.over_sampling import SMOTE
import itertools
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px
import time

# Sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.metrics import roc_auc_score, plot_confusion_matrix, plot_roc_curve, roc_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.utils import resample

# Models
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB

In [4]:
train = pd.read_csv("../input/spaceship-titanic/train.csv")
test = pd.read_csv("../input/spaceship-titanic/test.csv")

print("Train set shape : ", train.shape)
print("Test set shape : ", test.shape)
train.head()

In [5]:
print("TRAIN SET MISSING VALUES : ")
print(train.isna().sum())
print("")
print("TEST SET MISSING VALUES : ")
print(test.isna().sum())

In [6]:
print(f"Duplicates in train set : {train.duplicated().sum()},({np.round(100*train.duplicated().sum()/len(train), 1)}%)")
print("")
print(f"Duplicates in test set : {test.duplicated().sum()},({np.round(100*test.duplicated().sum()/len(test), 1)}%)")

In [7]:
train.nunique()

In [8]:
train.dtypes

**EDA**

In [9]:
plt.figure(figsize=(6, 6))

train["Transported"].value_counts().plot.pie(explode=[0.1, 0.1], autopct="%1.1f%%", shadow=True, textprops={"fontsize":16}).set_title("Target distribution")

In [10]:
plt.figure(figsize=(10, 4))

sns.histplot(data=train, x="Age", hue="Transported", binwidth=1, kde=True)

plt.title("Age distribution")
plt.xlabel("Age (years)")

Continuous features(Expenditure)

In [11]:
exp_feats = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]

fig = plt.figure(figsize=(10, 20))
for i, var_name in enumerate(exp_feats):
    #Left plot
    ax=fig.add_subplot(5, 2, 2*i+1)
    sns.histplot(data=train, x=var_name, axes=ax, bins=30, kde=False, hue="Transported")
    ax.set_title(var_name)
    
    #Right plot
    ax=fig.add_subplot(5, 2, 2*i+2)
    sns.histplot(data=train, x=var_name, axes=ax, bins=30, kde=True, hue="Transported")
    plt.ylim([0, 100])
    ax.set_title(var_name)
    
fig.tight_layout()
plt.show()


Categorical features

In [12]:
cat_feats = ["HomePlanet", "CryoSleep", "Destination", "VIP"]

fig = plt.figure(figsize=(10, 16))
for i, var_name in enumerate(cat_feats):
    ax = fig.add_subplot(4, 1, i+1)
    sns.countplot(data=train, x=var_name, axes=ax, hue="Transported")
    ax.set_title(var_name)
fig.tight_layout()
plt.show()

Qualitative features

In [13]:
qual_feats = ["PassengerId", "Cabin", "Name"]

train[qual_feats].head()

**Feature Engineering**

Age status

In [14]:
train["Age_group"] = np.nan
train.loc[train["Age"] <= 12, "Age_group"] = "Age_0-12"
train.loc[(train["Age"] > 12) & (train["Age"] < 18), "Age_group"] = "Age_13-17"
train.loc[(train["Age"] >= 18) & (train["Age"] <= 25), "Age_group"] = "Age_18-25"
train.loc[(train["Age"] > 25) & (train["Age"] <= 30), "Age_group"] = "Age_26-30"
train.loc[(train["Age"] > 30) & (train["Age"] <= 50), "Age_group"] = "Age_31-50"
train.loc[train["Age"] > 50, "Age_group"] = "Age_51+"

test["Age_group"] = np.nan
test.loc[test["Age"] <= 12, "Age_group"] = "Age_0-12"
test.loc[(test["Age"] > 12) & (test["Age"] < 18), "Age_group"] = "Age_13-17"
test.loc[(test["Age"] >= 18) & (test["Age"] <= 25), "Age_group"] = "Age_18-25"
test.loc[(test["Age"] > 25) & (test["Age"] <= 30), "Age_group"] = "Age_26-30"
test.loc[(test["Age"] > 30) & (test["Age"] <= 50), "Age_group"] = "Age_31-50"
test.loc[test["Age"] > 50, "Age_group"] = "Age_51+"

plt.figure(figsize=(10, 4))
g = sns.countplot(data=train, x="Age_group", hue="Transported",
                 order=["Age_0-12", "Age_13-17", "Age_18-25", "Age_26-30", "Age_31-50",
                       "Age_51+"])
plt.title("Age group distribution")

Expenditure

In [15]:
train["Expenditure"] = train[exp_feats].sum(axis=1)
train["No_spending"] = (train["Expenditure"]==0).astype(int)

test["Expenditure"] = test[exp_feats].sum(axis=1)
test["No_spending"] = (test["Expenditure"]==0).astype(int)

fig = plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
sns.histplot(data=train, x="Expenditure", hue="Transported", bins=200)
plt.title("Total expenditure (truncated)")
plt.ylim([0, 200])
plt.xlim([0, 20000])

plt.subplot(1, 2, 2)
sns.countplot(data=train, x="No_spending", hue="Transported")
plt.title("No spending indicator")
fig.tight_layout()

Passenger group

In [16]:
train["Group"] = train["PassengerId"].apply(lambda x: x.split("_")[0]).astype(int)
test["Group"] = test["PassengerId"].apply(lambda x: x.split("_")[0]).astype(int)

train["Group_size"] = train["Group"].map(lambda x: pd.concat([train["Group"], test["Group"]]).value_counts()[x])
test["Group_size"] = test["Group"].map(lambda x: pd.concat([train["Group"], test["Group"]]).value_counts()[x])

plt.figure(figsize=(20, 4))
plt.subplot(1, 2, 1)
sns.histplot(data=train, x="Group", hue="Transported", binwidth=1)
plt.title("Group")

plt.subplot(1, 2, 2)
sns.countplot(data=train, x="Group_size", hue="Transported")
plt.title("Group_size")
fig.tight_layout()

In [17]:
train["Solo"] = (train["Group_size"] == 1).astype(int)
test["Solo"] = (test["Group_size"] == 1).astype(int)

plt.figure(figsize=(10, 4))
sns.countplot(data=train, x="Solo", hue="Transported")
plt.title("Passenger travelling solo or not")
plt.ylim([0, 3000])

In [23]:
train["Cabin"].fillna("Z/9999/Z", inplace=True)
test["Cabin"].fillna("Z/9999/Z", inplace=True)

train["Cabin_deck"] = train["Cabin"].apply(lambda x: x.split("/")[0])
train["Cabin_number"] = train["Cabin"].apply(lambda x: x.split("/")[1]).astype(int)
train["Cabin_side"] = train["Cabin"].apply(lambda x: x.split("/")[2])

test["Cabin_deck"] = test["Cabin"].apply(lambda x: x.split("/")[0])
test["Cabin_number"] = test["Cabin"].apply(lambda x: x.split("/")[1]).astype(int)
test["Cabin_side"] = test["Cabin"].apply(lambda x: x.split("/")[2])

train.loc[train["Cabin_deck"] == "Z", "Cabin_deck"] = np.nan
train.loc[train["Cabin_number"] == 9999, "Cabin_number"] = np.nan
train.loc[train["Cabin_side"] == "Z", "Cabin_side"] == np.nan
test.loc[test["Cabin_deck"] == "Z", "Cabin_deck"] = np.nan
test.loc[test["Cabin_number"] == 9999, "Cabin_number"] = np.nan
test.loc[test["Cabin_side"] == "Z", "Cabin_side"] == np.nan

train.drop("Cabin", axis=1, inplace=True)
test.drop("Cabin", axis=1, inplace=True)

fig = plt.figure(figsize=(10, 12))
plt.subplot(3, 1, 1)
sns.countplot(data=train, x="Cabin_deck", hue="Transported", order=["A", "B", "C", "D", "E", "F", "G", "T"])
plt.title("Cabin deck")

plt.subplot(3, 1, 2)
sns.histplot(data=train, x="Cabin_number", hue="Transported", binwidth=20)
plt.vlines(300, ymin=0, ymax=200, color="black")
plt.vlines(600, ymin=0, ymax=200, color="black")
plt.vlines(900, ymin=0, ymax=200, color="black")
plt.vlines(1200, ymin=0, ymax=200, color="black")
plt.vlines(1500, ymin=0, ymax=200, color="black")
plt.vlines(1800, ymin=0, ymax=200, color="black")
plt.title("Cabin number")
plt.xlim([0, 2000])

plt.subplot(3, 1, 3)
sns.countplot(data=train, x="Cabin_side", hue="Transported")
plt.title("Cabin side")
fig.tight_layout()


In [ ]:
train["Cabin_region1"] = (train["Cabin_number"] < 300).astype(int)
train["Cabin_region2"] = ((train["Cabin_number"] >= 300) & (train["Cabin_number"] < 600)).astype(int)